In [1]:
import pandas as pd
import numpy as np

In [2]:
# Clean raw dengue data
df_dengue = pd.DataFrame(columns=['Cases'])
for year in range(2009, 2018):
    temp = pd.read_csv(f'../data/raw/{year}_dengue.csv')
    # drop NaN rows
    temp = temp[temp.iloc[:,0].notna()]
    # use last row (total as opposed to district data)
    temp = temp.iloc[-1]
    temp = pd.DataFrame(temp[1:-1])
    temp.columns = ['Cases']
    index = pd.date_range(start=f'1/1/{year}', periods=52, freq='W')
    temp['index'] = index
    temp = temp.set_index('index')
    df_dengue = pd.concat([df_dengue, temp])
df_dengue

,Cases
2009-01-04,67.0
2009-01-11,50.0
2009-01-18,46.0
2009-01-25,67.0
2009-02-01,60.0
...,...
2017-11-26,20.0
2017-12-03,29.0
2017-12-10,29.0
2017-12-17,22.0


In [3]:
df_dengue.to_csv('../data/clean/dengue_cases.csv')

In [4]:
df_weather = pd.read_csv('../data/raw/davao_weather_data.csv')
df_weather

,YEAR,MONTH,DAY,RAINFALL,TMAX,TMIN,TMEAN,RH,STA. PRESS,WIND SPEED,WIND DIRECTION
0,2008,1,1,1.2,31.8,23.9,27.9,86,1006.1,2,360
1,2008,1,2,11,31.0,24.1,27.5,88,1006.9,2,360
2,2008,1,3,9.4,31.6,23.7,27.7,83,1007.2,3,20
3,2008,1,4,11.6,29.7,23.2,26.5,90,1008.1,2,40
4,2008,1,5,0,30.3,23.8,27.1,82,1008.5,3,20
...,...,...,...,...,...,...,...,...,...,...,...
3648,2017,12,27,1.6,29.7,25.0,27.4,84,7.9,2,20
3649,2017,12,28,0,32.0,25.0,28.5,79,7.6,2,20
3650,2017,12,29,0,32.0,24.0,28.0,77,8.1,2,20
3651,2017,12,30,0,33.2,24.9,29.1,73,6.5,2,20


In [5]:
# Use only 2009+ data to match with df_dengue
df_weather = df_weather[df_weather.YEAR >= 2009]
df_weather

,YEAR,MONTH,DAY,RAINFALL,TMAX,TMIN,TMEAN,RH,STA. PRESS,WIND SPEED,WIND DIRECTION
366,2009,1,1,1.4,31.7,23.6,27.7,83,1007.2,2,40
367,2009,1,2,0.5,31.4,24.8,28.1,83,1007.2,2,360
368,2009,1,3,0,31.1,23.9,27.5,82,1007.7,3,20
369,2009,1,4,0,31.3,24.3,27.8,83,1007.4,2,40
370,2009,1,5,35.6,33.0,24.8,28.9,82,1007.1,2,70
...,...,...,...,...,...,...,...,...,...,...,...
3648,2017,12,27,1.6,29.7,25.0,27.4,84,7.9,2,20
3649,2017,12,28,0,32.0,25.0,28.5,79,7.6,2,20
3650,2017,12,29,0,32.0,24.0,28.0,77,8.1,2,20
3651,2017,12,30,0,33.2,24.9,29.1,73,6.5,2,20


In [6]:
df_weather.describe()

,YEAR,MONTH,DAY,TMAX,TMIN,TMEAN,RH,STA. PRESS,WIND SPEED,WIND DIRECTION
count,3287.000000,3287.000000,3287.000000,3287.000000,3287.000000,3287.000000,3287.000000,3287.000000,3287.000000,3287.000000
mean,2013.000608,6.523274,15.728628,31.958138,24.419957,28.207058,81.276848,896.352936,1.871311,208.700943
std,2.582185,3.449133,8.801004,3.660796,2.697985,3.105700,5.250397,314.252299,0.661654,121.180982
min,2009.000000,1.000000,1.000000,-2.000000,-2.000000,-2.000000,60.000000,1.900000,0.000000,0.000000
25%,2011.000000,4.000000,8.000000,31.550000,24.000000,27.900000,78.000000,1005.700000,1.000000,140.000000
50%,2013.000000,7.000000,16.000000,32.600000,24.600000,28.600000,82.000000,1007.100000,2.000000,180.000000
75%,2015.000000,10.000000,23.000000,33.300000,25.200000,29.200000,85.000000,1008.400000,2.000000,360.000000
max,2017.000000,12.000000,31.000000,38.600000,28.000000,32.200000,97.000000,1013.000000,4.000000,360.000000


In [7]:
from datetime import date
df_weather['date'] = df_weather.apply(lambda x: date(int(x['YEAR']), int(x['MONTH']), int(x['DAY'])), axis = 1)
df_weather['date'] = pd.to_datetime(df_weather['date'])
df_weather.set_index('date', inplace=True)
df_weather = df_weather.drop(columns=['YEAR', 'MONTH', 'DAY'])
df_weather

<ipython-input-7-cd638dcff162>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather['date'] = df_weather.apply(lambda x: date(int(x['YEAR']), int(x['MONTH']), int(x['DAY'])), axis = 1)
<ipython-input-7-cd638dcff162>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather['date'] = pd.to_datetime(df_weather['date'])


,RAINFALL,TMAX,TMIN,TMEAN,RH,STA. PRESS,WIND SPEED,WIND DIRECTION
date,,,,,,,,
2009-01-01,1.4,31.7,23.6,27.7,83,1007.2,2,40
2009-01-02,0.5,31.4,24.8,28.1,83,1007.2,2,360
2009-01-03,0,31.1,23.9,27.5,82,1007.7,3,20
2009-01-04,0,31.3,24.3,27.8,83,1007.4,2,40
2009-01-05,35.6,33.0,24.8,28.9,82,1007.1,2,70
...,...,...,...,...,...,...,...,...
2017-12-27,1.6,29.7,25.0,27.4,84,7.9,2,20
2017-12-28,0,32.0,25.0,28.5,79,7.6,2,20
2017-12-29,0,32.0,24.0,28.0,77,8.1,2,20


In [8]:
# Fix RAINFALL column
def is_float(string):
  try:
    float(string)
    return True
  except ValueError:  # String is not a number
    return False

df_weather[~df_weather['RAINFALL'].map(lambda x: is_float(x))]

,RAINFALL,TMAX,TMIN,TMEAN,RH,STA. PRESS,WIND SPEED,WIND DIRECTION
date,,,,,,,,
2015-01-13,T,31.4,24.5,28.0,81,1009.0,2,360
2015-01-18,T,34.0,26.0,30.0,78,1008.6,2,180
2015-01-20,T,33.0,25.3,29.2,78,1011.4,2,360
2015-02-19,T,30.9,23.8,27.4,76,1010.0,2,360
2015-02-20,T,32.1,24.0,28.1,75,1010.0,3,360
...,...,...,...,...,...,...,...,...
2017-11-07,T,31.8,24.9,28.4,82,5.7,2,20
2017-11-12,T,29.3,23.9,26.6,83,6.0,2,20
2017-11-28,T,34.0,25.8,29.9,75,5.5,3,360


### From documents given by PAGASA, T  = Trace amount (<0.01 mL)

In [9]:
def fix_float(string):
    try:    
        return float(string)
    except ValueError:  # String T - trace amounts ~0 mL
        return 0
df_weather['RAINFALL'] = df_weather['RAINFALL'].map(lambda x: fix_float(x))
df_weather

,RAINFALL,TMAX,TMIN,TMEAN,RH,STA. PRESS,WIND SPEED,WIND DIRECTION
date,,,,,,,,
2009-01-01,1.4,31.7,23.6,27.7,83,1007.2,2,40
2009-01-02,0.5,31.4,24.8,28.1,83,1007.2,2,360
2009-01-03,0.0,31.1,23.9,27.5,82,1007.7,3,20
2009-01-04,0.0,31.3,24.3,27.8,83,1007.4,2,40
2009-01-05,35.6,33.0,24.8,28.9,82,1007.1,2,70
...,...,...,...,...,...,...,...,...
2017-12-27,1.6,29.7,25.0,27.4,84,7.9,2,20
2017-12-28,0.0,32.0,25.0,28.5,79,7.6,2,20
2017-12-29,0.0,32.0,24.0,28.0,77,8.1,2,20


### TMAX, TMIN, TMEAN has values of -2 which means day was not recorded
the whole month of september 2017 has no temperature

**imputation**, use the values from september 2016

In [10]:
df_weather[(df_weather.TMAX == -2) | (df_weather.TMIN == -2) | (df_weather.TMEAN == -2)]

,RAINFALL,TMAX,TMIN,TMEAN,RH,STA. PRESS,WIND SPEED,WIND DIRECTION
date,,,,,,,,
2017-09-01,0.2,-2.0,-2.0,-2.0,77,7.8,2,180
2017-09-02,0.8,-2.0,-2.0,-2.0,78,7.9,3,180
2017-09-03,5.8,-2.0,-2.0,-2.0,77,8.2,2,180
2017-09-04,0.0,-2.0,-2.0,-2.0,79,8.2,1,180
2017-09-05,2.2,-2.0,-2.0,-2.0,76,7.9,2,180
2017-09-06,0.4,-2.0,-2.0,-2.0,76,7.7,2,180
2017-09-07,40.8,-2.0,-2.0,-2.0,82,8.1,2,200
2017-09-08,0.0,-2.0,-2.0,-2.0,80,7.4,1,200
2017-09-09,19.2,-2.0,-2.0,-2.0,78,5.9,2,180


In [11]:
df_weather.loc['2017-09-01':'2017-09-30', ['TMAX', 'TMIN', 'TMEAN']] = df_weather.loc['2016-09-01':'2016-09-30', ['TMAX', 'TMIN', 'TMEAN']].values
df_weather.loc['2017-09-01':'2017-09-30', ['TMAX', 'TMIN', 'TMEAN']]

,TMAX,TMIN,TMEAN
date,,,
2017-09-01,34.1,25.2,29.7
2017-09-02,34.3,26.0,30.2
2017-09-03,34.2,24.6,29.4
2017-09-04,34.0,25.7,29.9
2017-09-05,33.6,24.6,29.1
2017-09-06,33.5,25.0,29.3
2017-09-07,32.1,24.1,28.1
2017-09-08,33.2,25.2,29.2
2017-09-09,33.5,26.0,29.8


### NOTE: temporarily dropping WIND DIRECTION and STA. PRESS for the sake of simplicity
- possible resouce for wind direction: https://www.kaggle.com/coryjog/awea-intro-to-wind-analysis-with-pandas

In [12]:
df_weather = df_weather.drop(columns=['WIND DIRECTION', 'STA. PRESS'])
df_weather

,RAINFALL,TMAX,TMIN,TMEAN,RH,WIND SPEED
date,,,,,,
2009-01-01,1.4,31.7,23.6,27.7,83,2
2009-01-02,0.5,31.4,24.8,28.1,83,2
2009-01-03,0.0,31.1,23.9,27.5,82,3
2009-01-04,0.0,31.3,24.3,27.8,83,2
2009-01-05,35.6,33.0,24.8,28.9,82,2
...,...,...,...,...,...,...
2017-12-27,1.6,29.7,25.0,27.4,84,2
2017-12-28,0.0,32.0,25.0,28.5,79,2
2017-12-29,0.0,32.0,24.0,28.0,77,2


In [13]:
df_weekly = df_weather.resample('W', ).agg({'RAINFALL': ['min', 'max', np.mean], 'TMAX': 'max', 'TMIN': 'min', 'TMEAN': np.mean, 'RH': ['min', 'max', np.mean], 'WIND SPEED': np.mean})
df_weekly.index = pd.to_datetime(df_weekly.index)
df_weekly

RAINFALL                    TMAX  TMIN      TMEAN  RH      \
                min    max       mean   max   min       mean min max   
date                                                                   
2009-01-04      0.0    1.4   0.475000  31.7  23.6  27.775000  82  83   
2009-01-11      0.0   35.6  12.142857  33.0  23.5  28.385714  82  86   
2009-01-18      0.0   54.0  14.357143  30.7  22.3  25.771429  87  96   
2009-01-25      0.0   19.2   2.742857  33.0  23.7  28.385714  80  87   
2009-02-01      0.0   24.8   4.028571  33.0  23.3  28.100000  77  87   
...             ...    ...        ...   ...   ...        ...  ..  ..   
2017-12-03      0.0    0.4   0.057143  34.1  24.2  29.528571  74  80   
2017-12-10      0.0  109.8  24.342857  35.0  24.4  29.457143  72  81   
2017-12-17      0.0   31.0  10.142857  34.4  24.0  28.328571  68  89   
2017-12-24      0.0   39.4   8.828571  34.0  23.6  28.528571  70  86   
2017-12-31      0.0   22.4   3.428571  33.6  24.0  28.571429  73  84   

                      WIND SPEED  
                 mean       mean  
date                              
2009-01-04  82.750000   2.250000  
2009-01-11  83.857143   2.000000  
2009-01-18  90.428571   2.714286  
2009-01-25  83.428571   1.571429  
2009-02-01  82.285714   2.285714  
...               ...        ...  
2017-12-03  76.571429   2.142857  
2017-12-10  76.000000   1.857143  
2017-12-17  78.000000   1.285714  
2017-12-24  79.428571   1.714286  
2017-12-31  77.857143   2.000000  

[470 rows x 10 columns]

### note that df_weekly = 470 rows, df_dengue = 468 rows

In [14]:
df_weekly[~df_weekly.index.isin(df_weekly.index.intersection(df_dengue.index).to_list())]

RAINFALL                  TMAX  TMIN      TMEAN  RH                 \
                min   max      mean   max   min       mean min max       mean   
date                                                                            
2012-12-30      0.0  28.8  4.314286  33.3  23.8  28.128571  75  88  81.571429   
2017-12-31      0.0  22.4  3.428571  33.6  24.0  28.571429  73  84  77.857143   

           WIND SPEED  
                 mean  
date                   
2012-12-30   1.571429  
2017-12-31   2.000000


### let's combine these indeces with the previous week

In [19]:
import statistics
fix_these_weeks = ['2012-12-30', '2017-12-31']


for week in fix_these_weeks:
    idx = df_weekly.index.get_loc(week)
    print(week)
    curr_row = df_weekly.loc[week]
    prev_row = df_weekly.iloc[idx - 1]
    new_row = pd.DataFrame([
        (min(curr_row['RAINFALL']['min'], prev_row['RAINFALL']['min'])),
        (max(curr_row['RAINFALL']['max'], prev_row['RAINFALL']['max'])),
        (statistics.mean([curr_row['RAINFALL']['mean'], prev_row['RAINFALL']['mean']])),
        (max(curr_row['TMAX']['max'], prev_row['TMAX']['max'])),
        (min(curr_row['TMIN']['min'], prev_row['TMIN']['min'])),
        (statistics.mean([curr_row['TMEAN']['mean'], prev_row['TMEAN']['mean']])),
        (min(curr_row['RH']['min'], prev_row['RH']['min'])),
        (max(curr_row['RH']['max'], prev_row['RH']['max'])),
        (statistics.mean([curr_row['RH']['mean'], prev_row['RH']['mean']])),
        (statistics.mean([curr_row['WIND SPEED']['mean'], prev_row['WIND SPEED']['mean']]))
    ]).transpose()
    new_row.columns = [['RAINFALL', 'RAINFALL', 'RAINFALL', 'TMAX', 'TMIN', 'TMEAN', 'RH', 'RH', 'RH', 'WIND SPEED'], 
                       ['min', 'max', 'mean', 'max', 'min', 'mean', 'min', 'max', 'mean', 'mean']]  
    df_weekly.iloc[idx - 1] = new_row
    df_weekly = df_weekly.drop(week)

2012-12-30


KeyError: "['2012-12-30'] not found in axis"

In [18]:
df_weekly

RAINFALL                    TMAX  TMIN      TMEAN    RH        \
                min    max       mean   max   min       mean   min   max   
date                                                                       
2009-01-04      0.0    1.4   0.475000  31.7  23.6  27.775000  82.0  83.0   
2009-01-11      0.0   35.6  12.142857  33.0  23.5  28.385714  82.0  86.0   
2009-01-18      0.0   54.0  14.357143  30.7  22.3  25.771429  87.0  96.0   
2009-01-25      0.0   19.2   2.742857  33.0  23.7  28.385714  80.0  87.0   
2009-02-01      0.0   24.8   4.028571  33.0  23.3  28.100000  77.0  87.0   
...             ...    ...        ...   ...   ...        ...   ...   ...   
2017-12-03      0.0    0.4   0.057143  34.1  24.2  29.528571  74.0  80.0   
2017-12-10      0.0  109.8  24.342857  35.0  24.4  29.457143  72.0  81.0   
2017-12-17      0.0   31.0  10.142857  34.4  24.0  28.328571  68.0  89.0   
2017-12-24      0.0   39.4   8.828571  34.0  23.6  28.528571  70.0  86.0   
2017-12-31      0.0   22.4   3.428571  33.6  24.0  28.571429  73.0  84.0   

                      WIND SPEED  
                 mean       mean  
date                              
2009-01-04  82.750000   2.250000  
2009-01-11  83.857143   2.000000  
2009-01-18  90.428571   2.714286  
2009-01-25  83.428571   1.571429  
2009-02-01  82.285714   2.285714  
...               ...        ...  
2017-12-03  76.571429   2.142857  
2017-12-10  76.000000   1.857143  
2017-12-17  78.000000   1.285714  
2017-12-24  79.428571   1.714286  
2017-12-31  77.857143   2.000000  

[470 rows x 10 columns]

In [ ]:
df_weekly.to_csv('../data/clean/weather_data.csv')